# Superdense Coding Implementation

In this lab, we'll implement the superdense coding protocol, which allows Alice to send 2 classical bits to Bob using only 1 qubit (when they share an entangled pair).


In [ ]:
# Import required libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# For reproducibility
import numpy as np
np.random.seed(42)

## Part 2: Specify the Bits to Transmit

Let's define the two classical bits (c and d) that Alice wants to send to Bob.
Try changing these values to see that the protocol works correctly!

In [ ]:
# Bits to be transmitted (you can change these!)
c = 1  # First bit
d = 0  # Second bit

print(f"Alice wants to send: c={c}, d={d}")

## Part 3: Build the Superdense Coding Circuit

Now we'll construct the quantum circuit:
1. **Entanglement Creation**: Create a Bell state between Alice's and Bob's qubits
2. **Encoding**: Alice applies gates based on bits c and d:
   - (c=0, d=0): Do nothing
   - (c=0, d=1): Apply X gate
   - (c=1, d=0): Apply Z gate
   - (c=1, d=1): Apply Z then X gate
3. **Decoding**: Bob performs Bell measurement to extract both bits

In [ ]:
# Create quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Step 1: Create Bell state (entanglement)
# q0 is Alice's qubit, q1 is Bob's qubit
qc.h(0)        # Apply Hadamard to q0
qc.cx(0, 1)    # Apply CNOT with q0 as control, q1 as target
qc.barrier()   # Visual separator

# Step 2: Alice's encoding based on bits c and d
if d == 1:
    qc.x(0)    # Apply X if d=1
if c == 1:
    qc.z(0)    # Apply Z if c=1
qc.barrier()   # Visual separator

# Step 3: Bob's decoding (Bell measurement)
qc.cx(0, 1)    # CNOT
qc.h(0)        # Hadamard
qc.barrier()   # Visual separator

# Step 4: Measure all qubits
qc.measure_all()

# Display the circuit
print("\nSuperdense Coding Circuit:")
print(qc.draw(output='text'))

# Display with matplotlib for better visualization
qc.draw(output='mpl')
plt.title(f"Superdense Coding: Transmitting c={c}, d={d}")
plt.show()

## Part 4: Run the Simulation

Let's run our circuit using the Qiskit Aer simulator to verify the protocol works.

In [ ]:
# Run the circuit on the Aer simulator
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc, shots=1024)
result = job.result()
counts = result.get_counts(qc)

# Display results
print(f"\nMeasurement results for c={c}, d={d}:")
print(counts)

# Plot the histogram
plot_histogram(counts)
plt.title(f"Results: Expected output = {c}{d}")
plt.show()

# Verify the result
expected = f"{c}{d}"
measured = list(counts.keys())[0]
print(f"\nExpected: {expected}")
print(f"Measured: {measured}")
print(f"Success: {expected == measured}")

## Part 5: Random Bit Generation (Fun Extension!)

Now let's use an additional qubit as a random bit generator to randomly choose c and d, 
then run the superdense coding protocol to verify that these bits are transmitted correctly.

In [ ]:
# Create a circuit with 4 qubits:
# q0, q1: random bit generators for c and d
# q2, q3: superdense coding qubits
qc_random = QuantumCircuit(4, 4)

# Generate random bits for c (using q0) and d (using q1)
qc_random.h(0)  # Superposition for c
qc_random.h(1)  # Superposition for d
qc_random.barrier()

# Measure the random bits
qc_random.measure(0, 0)  # Measure q0 -> c0 (this will be 'c')
qc_random.measure(1, 1)  # Measure q1 -> c1 (this will be 'd')
qc_random.barrier()

# Create Bell state for superdense coding (q2 and q3)
qc_random.h(2)
qc_random.cx(2, 3)
qc_random.barrier()

# Alice's encoding: Apply gates conditionally based on measured c and d
# If c0 (bit c) is 1, apply Z to q2
qc_random.z(2).c_if(0, 1)
# If c1 (bit d) is 1, apply X to q2
qc_random.x(2).c_if(1, 1)
qc_random.barrier()

# Bob's decoding
qc_random.cx(2, 3)
qc_random.h(2)
qc_random.barrier()

# Measure the superdense coding qubits
qc_random.measure(2, 2)  # Bob measures q2 -> c2
qc_random.measure(3, 3)  # Bob measures q3 -> c3

# Display the circuit
print("Random Superdense Coding Circuit:")
print(qc_random.draw(output='text'))

# Visualize
qc_random.draw(output='mpl')
plt.title("Superdense Coding with Random Bit Generation")
plt.show()

## Part 6: Run and Verify Random Protocol

Let's run the random version multiple times to verify that Alice's bits (c, d) 
always match Bob's received bits.

In [ ]:
# Run the random circuit
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc_random, shots=1000)
result = job.result()
counts = result.get_counts(qc_random)

# Display results
print("\nRandom Superdense Coding Results:")
print("Format: c3 c2 c1 c0 where:")
print("  c0 c1 = Alice's random bits (c, d)")
print("  c2 c3 = Bob's received bits")
print("\n", counts)

# Plot histogram
plot_histogram(counts)
plt.title("Random Superdense Coding: Alice and Bob's bits should match")
plt.show()

# Verify that Alice and Bob's bits always agree
print("\nVerification:")
all_match = True
for outcome, count in counts.items():
    # outcome format: "c3 c2 c1 c0"
    bits = outcome.split()
    alice_c = bits[3]  # c0
    alice_d = bits[2]  # c1
    bob_c = bits[1]    # c2
    bob_d = bits[0]    # c3
    
    match = (alice_c == bob_c and alice_d == bob_d)
    if count > 0:
        print(f"Alice sent: {alice_c}{alice_d}, Bob received: {bob_c}{bob_d} - Match: {match}")
        all_match = all_match and match

print(f"\nAll transmissions successful: {all_match}")

## Summary

1. **Superdense coding** allows sending 2 classical bits using 1 qubit (when sharing entanglement)
2. The protocol uses **Bell states** and conditional operations based on the bits to transmit
3. The encoding scheme:
   - (0,0): Identity (do nothing)
   - (0,1): Apply X gate
   - (1,0): Apply Z gate
   - (1,1): Apply ZX gates
4. Bob performs a **Bell measurement** to decode both bits

